In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#benign = 1 and malignant = 0
dataset = pd.read_csv('Breast_cancer_data[1].csv')
dataset = dataset.sort_values(by='mean_area')
dataset.head(10)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
101,6.981,13.43,43.79,143.5,0.11700,1
539,7.691,25.44,48.34,170.4,0.08668,1
538,7.729,25.49,47.98,178.8,0.08098,1
568,7.760,24.54,47.92,181.0,0.05263,1
46,8.196,16.84,51.71,201.9,0.08600,1
151,8.219,20.70,53.27,203.9,0.09405,1
314,8.597,18.60,54.09,221.2,0.10740,1
525,8.571,13.10,54.53,221.3,0.10360,1
61,8.598,20.98,54.66,221.8,0.12430,1
59,8.618,11.79,54.34,224.5,0.09752,1


In [3]:
dataset.isnull().sum()

mean_radius        0
mean_texture       0
mean_perimeter     0
mean_area          0
mean_smoothness    0
diagnosis          0
dtype: int64

In [4]:
# stores all the column labels in the list except for last column
# which is our target variable.
m= dataset.shape[1]
header = dataset.columns
header = list(header)
print(header)
dataset.shape

['mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area', 'mean_smoothness', 'diagnosis']


(569, 6)

In [5]:
# dividing the dataset into 75% for training and 25% for testing 
idx = 3*dataset.shape[0]//4
train = dataset.iloc[:idx,:]
test = dataset.iloc[idx:,:]
print("last 5 rows of training dataset:\n\n",train.tail())
print("\nfirst 5 rows of testing dataset:\n\n",test.head())

last 5 rows of training dataset:

      mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
133        15.71         13.93           102.0      761.7          0.09462   
182        15.70         20.31           101.2      766.6          0.09597   
258        15.66         23.20           110.2      773.5          0.11090   
11         15.78         17.89           103.6      781.0          0.09710   
118        15.78         22.91           105.7      782.6          0.11550   

     diagnosis  
133          1  
182          0  
258          0  
11           0  
118          0  

first 5 rows of testing dataset:

      mean_radius  mean_texture  mean_perimeter  mean_area  mean_smoothness  \
13         15.85         23.95           103.7      782.7          0.08401   
375        16.17         16.07           106.3      788.5          0.09880   
330        16.03         15.51           105.8      793.2          0.09491   
10         16.02         23.24           102.7

In [6]:
train = np.array(train)
test = np.array(test)

In [7]:
def unique_values(dataset,col_index):
    unique = np.unique(dataset[:,col_index])
    return(unique)
    
#x is a portion of the dataset for which we find entropy
def count_label(x):
    cnt = {}
    for row in x:
        if row[-1] not in cnt:
            cnt[row[-1]]=0
        cnt[row[-1]]+=1
    return cnt


def entropy(x):
    cnt = count_label(x)
    total = 0
    for val in cnt.values():
        total += val
        
    ent = 0 #entropy value
    for val in cnt.values():
        ent += (val/total)*np.log(val/total)
    
    return -1*ent


# dataset_ent is the entropy of the entire dataset 
def info_gain(left,right,parent_ent):
    n = len(left) + len(right)
    
    left_ent=entropy(left)
    right_ent=entropy(right)
    
    weighted_avg = (len(left)/n)*left_ent + (len(right)/n)*right_ent
    
    return parent_ent-weighted_avg

    
    
class Node:
    def __init__(self,question=None,split_feature=None,left=None,right=None,label=None):
        self.question = question
        self.split_feature = split_feature
        self.left = left
        self.right = right
        self.label = label
    
    def is_leaf(self):
        if self.label is not None:
            return True
    
        
        
def partition(rows,question,column):
    left, right = [],[]
    for row in rows:
        if row[column]>=question:
            right.append(row)
        else:
            left.append(row)
    return np.array(left), np.array(right)


        
def best_split(rows,column):
    max_gain = 0 
    ques, attr = None,None
    left,right=[],[]
    parent_entropy = entropy(rows)
    for col in range(rows.shape[1]-1):
        if col in column:
            continue
        for row in range(rows.shape[0]):
            l,r = partition(rows,rows[row,col],col)
            ig = info_gain(l,r,parent_entropy)
            if max_gain < ig:
                max_gain = ig
                ques = rows[row,col]
                attr = col
    return ques,attr


def find_common_label(column):
    types = np.unique(column)
    group = {t:0 for t in types}
    for c in column:
        group[c]+=1
        
    most_common=None
    cnt=0
    for t in types:
        if cnt<group[t]:
            most_common = t
            cnt = group[t]
    return most_common


        
def define_root(rows,max_depth):
    root = build_tree(rows,max_depth,column=[])
    return root


    
def build_tree(rows,max_depth,column):
    ques, attr = best_split(rows,column)
    groups = None
    if attr is not None:
        column.append(attr)
        groups = np.unique(rows[:,-1])
        
    if max_depth==0 or attr is None or len(groups) == 1:
        common_label = find_common_label(rows[:,-1])
        #print(common_label)
        return Node(label = common_label)
    
    left_part,right_part = partition(rows,ques,attr)
    left = build_tree(left_part,max_depth-1,column)
    right = build_tree(right_part,max_depth-1,column)
    
    return Node(ques,attr,left,right)
        
    
    
    
def traverse_tree(test_row,node):
    if node.is_leaf():
        return node.label
    
    if test_row[node.split_feature]<node.question:
        return traverse_tree(test_row,node.left)
    else:
        return traverse_tree(test_row,node.right)
    

def predict(test_set,root):
    predictions=[]
    for test_row in test_set:
        predictions.append(traverse_tree(test_row,root))
    return predictions
    
    
    
def calAccuracy(pred, y):
    correct=0
    for i in range(len(pred)):
        if pred[i]==y[i]:
            correct+=1
    return correct*100/len(pred)
            

In [8]:
max_depth = 100
root = define_root(train,max_depth)
pred = predict(test,root)
print(pred)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [9]:
accuracy = calAccuracy(pred,test[:,-1])
print(f"model accuracy on test data: {accuracy}%")

model accuracy on test data: 95.8041958041958%
